## Usage

- python 3.7
- pipenv
- dotenv
- twitter API (and client library)

In [ ]:
import os
import twitter
from dotenv import load_dotenv

In [6]:
load_dotenv()

CONSUMER_KEY = os.getenv("CONSUMER_KEY")
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET")
ACCESS_TOKEN_KEY = os.getenv("ACCESS_TOKEN_KEY")
ACCESS_TOKEN_SECRET = os.getenv("ACCESS_TOKEN_SECRET")

# WARNING: do not commit this file to git with these values printed to a cell's output

In [ ]:
'''
api = twitter.Api(consumer_key=CONSUMER_KEY,
                  consumer_secret=CONSUMER_SECRET,
                  access_token_key=ACCESS_TOKEN_KEY,
                  access_token_secret=ACCESS_TOKEN_SECRET)
'''

In [ ]:
# some potential hashtags include #libspill and #auspol

## Plans

- if there are no cached tweets (or if some flag is passed)
- search for tweets with a given hashtag using 30 day api in batches of 100 tweets
- receive 100 tweets, filter them by some characteristics and stream the filtered tweets into a collection
- page forward and repeat
- when the collection contains N tweets (or if we reach the end of the 30 day feed?) then cease fetching
- cache the results so we can access them from a file and not run out of monthly requests...